Connect to Drive for Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Load Files

In [ ]:
import csv
import pandas as pd
import numpy as np
import gc
gc.collect()
Deposit_Acct_Join122016 = pd.read_csv('/content/drive/My Drive/Deposit_Acct_Join122016.txt', sep="|")
print (Deposit_Acct_Join122016.head(2))
Deposit_InstitutionDetails122016 = pd.read_csv('/content/drive/My Drive/Deposit_InstitutionDetails122016.txt', sep="|")
Deposit_InstitutionDetails122016 = Deposit_InstitutionDetails122016.drop(['inst_typ', 'cert_nbr', 'rssd_id', 'brnch_srv_typ', 'ho_uninumbr', 'web', 'phone', 'asset_sz', 'hd_offc', 'est_dt', 'rtng_nbr', 'zip', 'county', 'branches', 'institutiondeposits', 'branchdeposits', 'cnty_fps', 'state_fps', 'msa', 'cbsa', 'tm_zone'], axis = 1)
print (Deposit_InstitutionDetails122016.head(2))
depositRateData_2016_12 = pd.read_csv('/content/drive/My Drive/depositRateData_2016_12.txt', sep="|", dtype={'accountnumber': 'str', 'productcode': 'str', 'producttype': 'str', 'tierminimum': 'int', 'tiermaximum': 'int', 'termlength': 'int', 'rate': 'float', 'apy': 'float', 'surveydate': 'str'})
print (depositRateData_2016_12.head(2))


  acct_nbr_loc acct_nbr_rt prd_typ_join    eff_date
0   MI02000024  MI02000024      BPREMMM  2013-01-26
1   MI02000024  MI02000024       BRELMM  1999-11-15
     acct_nbr                                 inst_nm  ...         lon        lat
0  AK00100001  Wells Fargo Bank, National Association  ... -149.887488  61.195833
1  AK00100002              First National Bank Alaska  ... -149.883667  61.188210

[2 rows x 8 columns]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


  accountnumber productcode producttype  ...   rate   apy  surveydate
0    AK00100003      MM100K          MM  ...  0.145  0.15  2016-11-30
1    AK00100006      MM100K          MM  ...  0.050  0.05  2016-11-30

[2 rows x 10 columns]


In [ ]:
Deposit_Acct_Join122016['eff_date'] = pd.to_datetime(Deposit_Acct_Join122016.eff_date)

keep the latest rate setter based on 'acct_nbr_loc','prd_typ_join'

---



In [ ]:
c_maxes = Deposit_Acct_Join122016.groupby(['acct_nbr_loc', 'prd_typ_join']).eff_date.transform(max)
Deposit_Acct_Join122016 = Deposit_Acct_Join122016.loc[Deposit_Acct_Join122016.eff_date == c_maxes]


In [ ]:
print (len(depositRateData_2016_12.index))
print (len(Deposit_Acct_Join122016.index))

1778853
2954164


In [ ]:
gc.collect()

33

Remove duplicates based on 'accountnumber', 'productcode', 'surveydate' from Deposit Rate Data file

In [ ]:
unique_depositRateData_2016_12 = depositRateData_2016_12[~depositRateData_2016_12[['accountnumber', 'productcode', 'surveydate']].apply(frozenset, axis=1).duplicated()]
unique_depositRateData_2016_12 = unique_depositRateData_2016_12.rename(columns={"accountnumber": "acct_nbr_loc" , "producttype": "prd_typ_join"})
del unique_depositRateData_2016_12['apy']
unique_depositRateData_2016_12.head(2)

,acct_nbr_loc,productcode,prd_typ_join,tierminimum,tiermaximum,termlength,termfrequency,rate,surveydate
0,AK00100003,MM100K,MM,2500,9999999,0,NaN,0.145,2016-11-30
1,AK00100006,MM100K,MM,25000,9999999,0,NaN,0.050,2016-11-30


In [ ]:
print (len(depositRateData_2016_12.index))
print (len(unique_depositRateData_2016_12.index))

1778853
1778853


In [ ]:
Deposit_InstitutionDetails122016 = Deposit_InstitutionDetails122016.rename(columns={"acct_nbr": "acct_nbr_loc"})
Deposit_InstitutionDetails122016['uninumbr'] = Deposit_InstitutionDetails122016['uninumbr'].replace('0.0', np.nan)
Deposit_InstitutionDetails122016 = Deposit_InstitutionDetails122016.dropna(axis=0, subset=['uninumbr'])
Deposit_InstitutionDetails122016.head(2)

,acct_nbr_loc,inst_nm,uninumbr,address,city,state,lon,lat
0,AK00100001,"Wells Fargo Bank, National Association",10952.0,301 W Northern Lights Blvd,Anchorage,AK,-149.887488,61.195833
1,AK00100002,First National Bank Alaska,10345.0,101 W 36th Ave,Anchorage,AK,-149.883667,61.188210


In [ ]:
Deposit_InstitutionDetails122016.uninumbr.isna().sum()

0

In [ ]:
unique_Deposit_InstitutionDetails122016 = Deposit_InstitutionDetails122016[~Deposit_InstitutionDetails122016[['acct_nbr_loc', 'uninumbr']].apply(frozenset, axis=1).duplicated()]
unique_Deposit_InstitutionDetails122016.head(2)

,acct_nbr_loc,inst_nm,uninumbr,address,city,state,lon,lat
0,AK00100001,"Wells Fargo Bank, National Association",10952.0,301 W Northern Lights Blvd,Anchorage,AK,-149.887488,61.195833
1,AK00100002,First National Bank Alaska,10345.0,101 W 36th Ave,Anchorage,AK,-149.883667,61.188210


In [ ]:
print (len(Deposit_InstitutionDetails122016.index))
print (len(unique_Deposit_InstitutionDetails122016.index))

109212
109212


In [ ]:
unique_depositRateData_2016_12['surveydate'] = pd.to_datetime(unique_depositRateData_2016_12.surveydate)


Merge account join file and deposit rate data file based on 'acct_nbr_loc', 'prd_typ_join'

In [ ]:
rate_acctjoin_data = pd.merge(Deposit_Acct_Join122016,unique_depositRateData_2016_12, on=['acct_nbr_loc', 'prd_typ_join'], how ='inner')
rate_acctjoin_data_filterdate = rate_acctjoin_data[rate_acctjoin_data['surveydate'] >= rate_acctjoin_data['eff_date']]
print (len(rate_acctjoin_data_filterdate.index))
#rate_acctjoin_data_filterdate = rate_acctjoin_data_filterdate.sort_values(by=['productcode','acct_nbr_loc'])
#rate_acctjoin_data_filterdate = rate_acctjoin_data_filterdate.reset_index()
#del rate_acctjoin_data_filterdate['index']
rate_acctjoin_data_filterdate.head(2)

1772425


,acct_nbr_loc,acct_nbr_rt,prd_typ_join,eff_date,productcode,tierminimum,tiermaximum,termlength,termfrequency,rate,surveydate
0,MI02000024,MI02000024,CD,1999-11-15,24MCD10K,5000,9999999,24,M,1.19,2016-11-29
1,MI02000024,MI02000024,CD,1999-11-15,24MCD100K,5000,9999999,24,M,1.19,2016-11-29


In [ ]:
gc.collect()

22

In [ ]:
rate_acctjoin_data_filterdate.drop_duplicates(keep=False,inplace=True)
print (len(rate_acctjoin_data_filterdate.index))
rate_acctjoin_data_filterdate.head(2)

1772425


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,acct_nbr_loc,acct_nbr_rt,prd_typ_join,eff_date,productcode,tierminimum,tiermaximum,termlength,termfrequency,rate,surveydate
0,MI02000024,MI02000024,CD,1999-11-15,24MCD10K,5000,9999999,24,M,1.19,2016-11-29
1,MI02000024,MI02000024,CD,1999-11-15,24MCD100K,5000,9999999,24,M,1.19,2016-11-29


In [ ]:
rate_acctjoin_data_filterdate = rate_acctjoin_data_filterdate.drop(columns=['prd_typ_join', 'eff_date'],axis=1)

In [ ]:
pc_list = rate_acctjoin_data_filterdate['productcode'].unique().tolist()


In [ ]:
#key_sublists = [pc_list[i * 150:(i + 1) * 150] for i in range((len(pc_list) + 5 - 1) // 150 )]  
#key1 = np.array([key_sublists.iloc[0]].values)
#key1
key1 = pc_list[0:151]
key2 = pc_list[151:302]
key3 = pc_list[302:453]
key4 = pc_list[453:604]
key5 = pc_list[604:755]
pc_list_dict1 = {name: rate_acctjoin_data_filterdate.loc[rate_acctjoin_data_filterdate['productcode'] == name] for name in key1}
pc_list_dict2 = {name: rate_acctjoin_data_filterdate.loc[rate_acctjoin_data_filterdate['productcode'] == name] for name in key2}
pc_list_dict3 = {name: rate_acctjoin_data_filterdate.loc[rate_acctjoin_data_filterdate['productcode'] == name] for name in key3}
pc_list_dict4 = {name: rate_acctjoin_data_filterdate.loc[rate_acctjoin_data_filterdate['productcode'] == name] for name in key4}
pc_list_dict5 = {name: rate_acctjoin_data_filterdate.loc[rate_acctjoin_data_filterdate['productcode'] == name] for name in key5}


In [ ]:
len(key1)+len(key2)+len(key3)+len(key4)+len(key5)

748

In [ ]:
gc.collect()

0

In [ ]:
df_blank = unique_depositRateData_2016_12['acct_nbr_loc'].unique()
df_blank = pd.DataFrame(df_blank.flatten())
df_blank = df_blank.rename(columns={0: 'acct_nbr_loc'})
print((df_blank))
type(df_blank)

     acct_nbr_loc
0      AK00100003
1      AK00100006
2      AK00400001
3      AK00400002
4      AL00100010
...           ...
8552   GA00700155
8553   AL00900023
8554   IL07400009
8555   MI04900013
8556   ID00100029

[8557 rows x 1 columns]


pandas.core.frame.DataFrame

In [ ]:
i=1
df_blank1 = df_blank
import datetime
from functools import reduce
for key in pc_list_dict1:
  new_df = pd.DataFrame(pc_list_dict1[key])
  new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','productcode': key+'_productcode','surveydate': key+'_surveydate','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  #new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  #new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  df_blank1 = reduce(lambda x,y: pd.merge(x,y, on='acct_nbr_loc', how='outer'), [df_blank1, new_df])
  #gc.collect()
  #df_blank = pd.concat([df_blank,new_df], axis=1)
  #df_blank = df_blank.append(new_df, ignore_index=True)
  print('Appending product ',key,' data to the dataframe. Loop number: ',i,' of',len(key1),' time: ',datetime.datetime.utcnow())
  i=i+1
print(df_blank1)
wide1 = df_blank1.loc[:,~df_blank1.columns.duplicated()]
#wide1.to_csv("/content/drive/My Drive/key1.csv", index = False, sep='|',mode = 'w', encoding='utf-8')

Appending product  24MCD10K  data to the dataframe. Loop number:  1  of 151  time:  2020-02-21 14:23:45.739905
Appending product  24MCD100K  data to the dataframe. Loop number:  2  of 151  time:  2020-02-21 14:23:45.759465
Appending product  30MCD10K  data to the dataframe. Loop number:  3  of 151  time:  2020-02-21 14:23:45.777042
Appending product  24MCD40K  data to the dataframe. Loop number:  4  of 151  time:  2020-02-21 14:23:45.799574
Appending product  30MCD40K  data to the dataframe. Loop number:  5  of 151  time:  2020-02-21 14:23:45.820659
Appending product  09MCD100K  data to the dataframe. Loop number:  6  of 151  time:  2020-02-21 14:23:45.843311
Appending product  12MCD10K  data to the dataframe. Loop number:  7  of 151  time:  2020-02-21 14:23:45.870879
Appending product  12MCD100K  data to the dataframe. Loop number:  8  of 151  time:  2020-02-21 14:23:45.900628
Appending product  18MCD10K  data to the dataframe. Loop number:  9  of 151  time:  2020-02-21 14:23:45.93214

In [ ]:
gc.collect()

11

In [ ]:
i=1
df_blank2 = df_blank
#df_blank = df_blank.fillna(0)
import datetime
from functools import reduce
for key in pc_list_dict2:
  new_df = pd.DataFrame(pc_list_dict2[key])
  new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','productcode': key+'_productcode','surveydate': key+'_surveydate','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  #new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  #new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  df_blank2 = reduce(lambda x,y: pd.merge(x,y, on='acct_nbr_loc', how='outer'), [df_blank2, new_df])
  #gc.collect()
  #df_blank = pd.concat([df_blank,new_df], axis=1)
  #df_blank = df_blank.append(new_df, ignore_index=True)
  print('Appending product ',key,' data to the dataframe. Loop number: ',i,' of',len(key2),' time: ',datetime.datetime.utcnow())
  i=i+1
print(df_blank2)
wide2 = df_blank2.loc[:,~df_blank2.columns.duplicated()]
#wide2.to_csv("/content/drive/My Drive/key2.csv", index = False, sep='|',mode = 'w', encoding='utf-8')

Appending product  36MCD25K  data to the dataframe. Loop number:  1  of 151  time:  2020-02-21 14:24:10.711281
Appending product  FIXIRA50K  data to the dataframe. Loop number:  2  of 151  time:  2020-02-21 14:24:10.728738
Appending product  FIXIRA0K  data to the dataframe. Loop number:  3  of 151  time:  2020-02-21 14:24:10.747035
Appending product  INTCK5K  data to the dataframe. Loop number:  4  of 151  time:  2020-02-21 14:24:10.768282
Appending product  INTCK7.5K  data to the dataframe. Loop number:  5  of 151  time:  2020-02-21 14:24:10.790911
Appending product  INTCK8K  data to the dataframe. Loop number:  6  of 151  time:  2020-02-21 14:24:10.815311
Appending product  INTCK0K  data to the dataframe. Loop number:  7  of 151  time:  2020-02-21 14:24:10.841963
Appending product  INTCK2.5K  data to the dataframe. Loop number:  8  of 151  time:  2020-02-21 14:24:10.870610
Appending product  MM100K  data to the dataframe. Loop number:  9  of 151  time:  2020-02-21 14:24:10.900588
App

In [ ]:
gc.collect()

11

In [ ]:
i=1
df_blank3 = df_blank
#df_blank = df_blank.fillna(0)
import datetime
from functools import reduce
for key in pc_list_dict3:
  new_df = pd.DataFrame(pc_list_dict3[key])
  new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','productcode': key+'_productcode','surveydate': key+'_surveydate','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  #new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  #new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  df_blank3 = reduce(lambda x,y: pd.merge(x,y, on='acct_nbr_loc', how='outer'), [df_blank3, new_df])
  #gc.collect()
  #df_blank = pd.concat([df_blank,new_df], axis=1)
  #df_blank = df_blank.append(new_df, ignore_index=True)
  print('Appending product ',key,' data to the dataframe. Loop number: ',i,' of',len(key3),' time: ',datetime.datetime.utcnow())
  i=i+1
print(df_blank3)
wide3 = df_blank3.loc[:,~df_blank3.columns.duplicated()]
#wide3.to_csv("/content/drive/My Drive/key3.csv", index = False, sep='|',mode = 'w', encoding='utf-8')

Appending product  24MBSCD50K  data to the dataframe. Loop number:  1  of 151  time:  2020-02-21 14:24:36.496165
Appending product  36MBSCD250K  data to the dataframe. Loop number:  2  of 151  time:  2020-02-21 14:24:36.511419
Appending product  36MBSCD50K  data to the dataframe. Loop number:  3  of 151  time:  2020-02-21 14:24:36.526139
Appending product  48MBSCD250K  data to the dataframe. Loop number:  4  of 151  time:  2020-02-21 14:24:36.542355
Appending product  48MBSCD50K  data to the dataframe. Loop number:  5  of 151  time:  2020-02-21 14:24:36.562130
Appending product  60MBSCD250K  data to the dataframe. Loop number:  6  of 151  time:  2020-02-21 14:24:36.582057
Appending product  60MBSCD50K  data to the dataframe. Loop number:  7  of 151  time:  2020-02-21 14:24:36.602693
Appending product  60MBSCD10K  data to the dataframe. Loop number:  8  of 151  time:  2020-02-21 14:24:36.626539
Appending product  06MBSCD100K  data to the dataframe. Loop number:  9  of 151  time:  2020-0

In [ ]:
i=1
df_blank4 = df_blank
#df_blank = df_blank.fillna(0)
import datetime
from functools import reduce
for key in pc_list_dict4:
  new_df = pd.DataFrame(pc_list_dict4[key])
  new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','productcode': key+'_productcode','surveydate': key+'_surveydate','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  #new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  #new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  df_blank4 = reduce(lambda x,y: pd.merge(x,y, on='acct_nbr_loc', how='outer'), [df_blank4, new_df])
  #gc.collect()
  #df_blank = pd.concat([df_blank,new_df], axis=1)
  #df_blank = df_blank.append(new_df, ignore_index=True)
  print('Appending product ',key,' data to the dataframe. Loop number: ',i,' of',len(key4),' time: ',datetime.datetime.utcnow())
  i=i+1
print(df_blank4)
wide4 = df_blank4.loc[:,~df_blank4.columns.duplicated()]
#wide4.to_csv("/content/drive/My Drive/key4.csv", index = False, sep='|',mode = 'w', encoding='utf-8')

Appending product  PREMMM150K  data to the dataframe. Loop number:  1  of 151  time:  2020-02-21 14:24:59.255120
Appending product  PREMMM75K  data to the dataframe. Loop number:  2  of 151  time:  2020-02-21 14:24:59.268463
Appending product  PREMMM500K  data to the dataframe. Loop number:  3  of 151  time:  2020-02-21 14:24:59.283245
Appending product  PREMMM1MILL  data to the dataframe. Loop number:  4  of 151  time:  2020-02-21 14:24:59.299628
Appending product  PREMMM25K  data to the dataframe. Loop number:  5  of 151  time:  2020-02-21 14:24:59.319555
Appending product  PREMMM200K  data to the dataframe. Loop number:  6  of 151  time:  2020-02-21 14:24:59.340927
Appending product  PREMMM250K  data to the dataframe. Loop number:  7  of 151  time:  2020-02-21 14:24:59.362860
Appending product  PREMMM300K  data to the dataframe. Loop number:  8  of 151  time:  2020-02-21 14:24:59.387746
Appending product  PREMMM400K  data to the dataframe. Loop number:  9  of 151  time:  2020-02-21 

In [ ]:
i=1
df_blank5 = df_blank
#df_blank = df_blank.fillna(0)
import datetime
from functools import reduce
for key in pc_list_dict5:
  new_df = pd.DataFrame(pc_list_dict5[key])
  new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','productcode': key+'_productcode','surveydate': key+'_surveydate','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  #new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  #new_df = new_df.rename(columns={'acct_nbr_rt': key+'_acct_nbr_rt','tierminimum': key+'_tierminimum','tiermaximum': key+'_tiermaximum','termlength': key+'_termlength','termfrequency': key+'_termfrequency','rate': key+'_rate'})
  df_blank5 = reduce(lambda x,y: pd.merge(x,y, on='acct_nbr_loc', how='outer'), [df_blank5, new_df])
  #gc.collect()
  #df_blank = pd.concat([df_blank,new_df], axis=1)
  #df_blank = df_blank.append(new_df, ignore_index=True)
  print('Appending product ',key,' data to the dataframe. Loop number: ',i,' of',len(key5),' time: ',datetime.datetime.utcnow())
  i=i+1
print(df_blank5)
wide5 = df_blank5.loc[:,~df_blank5.columns.duplicated()]
#wide5.to_csv("/content/drive/My Drive/key5.csv", index = False, sep='|',mode = 'w', encoding='utf-8')

Appending product  36MRELCD10K  data to the dataframe. Loop number:  1  of 144  time:  2020-02-21 14:25:23.476759
Appending product  48MRELCD10K  data to the dataframe. Loop number:  2  of 144  time:  2020-02-21 14:25:23.490288
Appending product  15MRELCD10K  data to the dataframe. Loop number:  3  of 144  time:  2020-02-21 14:25:23.504486
Appending product  01MRELCD10K  data to the dataframe. Loop number:  4  of 144  time:  2020-02-21 14:25:23.519249
Appending product  03MRELCD90K  data to the dataframe. Loop number:  5  of 144  time:  2020-02-21 14:25:23.535263
Appending product  06MRELCD90K  data to the dataframe. Loop number:  6  of 144  time:  2020-02-21 14:25:23.552541
Appending product  07MRELCD10K  data to the dataframe. Loop number:  7  of 144  time:  2020-02-21 14:25:23.572900
Appending product  09MRELCD90K  data to the dataframe. Loop number:  8  of 144  time:  2020-02-21 14:25:23.595840
Appending product  10MRELCD10K  data to the dataframe. Loop number:  9  of 144  time:  2

In [ ]:
wide2.head()

,acct_nbr_loc,36MCD25K_acct_nbr_rt,36MCD25K_productcode,36MCD25K_tierminimum,36MCD25K_tiermaximum,36MCD25K_termlength,36MCD25K_termfrequency,36MCD25K_rate,36MCD25K_surveydate,FIXIRA50K_acct_nbr_rt,FIXIRA50K_productcode,FIXIRA50K_tierminimum,FIXIRA50K_tiermaximum,FIXIRA50K_termlength,FIXIRA50K_termfrequency,FIXIRA50K_rate,FIXIRA50K_surveydate,FIXIRA0K_acct_nbr_rt,FIXIRA0K_productcode,FIXIRA0K_tierminimum,FIXIRA0K_tiermaximum,FIXIRA0K_termlength,FIXIRA0K_termfrequency,FIXIRA0K_rate,FIXIRA0K_surveydate,INTCK5K_acct_nbr_rt,INTCK5K_productcode,INTCK5K_tierminimum,INTCK5K_tiermaximum,INTCK5K_termlength,INTCK5K_termfrequency,INTCK5K_rate,INTCK5K_surveydate,INTCK7.5K_acct_nbr_rt,INTCK7.5K_productcode,INTCK7.5K_tierminimum,INTCK7.5K_tiermaximum,INTCK7.5K_termlength,INTCK7.5K_termfrequency,INTCK7.5K_rate,...,12MBSCD250K_acct_nbr_rt,12MBSCD250K_productcode,12MBSCD250K_tierminimum,12MBSCD250K_tiermaximum,12MBSCD250K_termlength,12MBSCD250K_termfrequency,12MBSCD250K_rate,12MBSCD250K_surveydate,18MBSCD10K_acct_nbr_rt,18MBSCD10K_productcode,18MBSCD10K_tierminimum,18MBSCD10K_tiermaximum,18MBSCD10K_termlength,18MBSCD10K_termfrequency,18MBSCD10K_rate,18MBSCD10K_surveydate,18MBSCD50K_acct_nbr_rt,18MBSCD50K_productcode,18MBSCD50K_tierminimum,18MBSCD50K_tiermaximum,18MBSCD50K_termlength,18MBSCD50K_termfrequency,18MBSCD50K_rate,18MBSCD50K_surveydate,24MBSCD10K_acct_nbr_rt,24MBSCD10K_productcode,24MBSCD10K_tierminimum,24MBSCD10K_tiermaximum,24MBSCD10K_termlength,24MBSCD10K_termfrequency,24MBSCD10K_rate,24MBSCD10K_surveydate,24MBSCD250K_acct_nbr_rt,24MBSCD250K_productcode,24MBSCD250K_tierminimum,24MBSCD250K_tiermaximum,24MBSCD250K_termlength,24MBSCD250K_termfrequency,24MBSCD250K_rate,24MBSCD250K_surveydate
0,AK00100003,AK00100003,36MCD25K,10000.0,99999.0,36.0,M,1.64,2016-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,AK00100003,FIXIRA0K,500.0,9999.0,0.0,NaN,0.745,2016-11-30,AK00100003,INTCK5K,1000.0,9999999.0,0.0,NaN,0.05,2016-11-30,AK00100003,INTCK7.5K,1000.0,9999999.0,0.0,NaN,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,AK00100006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,AK00100006,FIXIRA50K,500.0,9999999.0,0.0,NaN,0.20,2016-11-30,AK00100006,FIXIRA0K,500.0,9999999.0,0.0,NaN,0.200,2016-11-30,AK00100006,INTCK5K,2501.0,9999999.0,0.0,NaN,0.04,2016-11-30,AK00100006,INTCK7.5K,2501.0,9999999.0,0.0,NaN,0.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,AK00400001,AK00400001,36MCD25K,25000.0,99999.0,36.0,M,1.00,2016-11-30,AK00400001,FIXIRA50K,25000.0,99999.0,0.0,NaN,0.65,2016-11-30,AK00400001,FIXIRA0K,2000.0,24999.0,0.0,NaN,0.550,2016-11-30,AK00400001,INTCK5K,1.0,24999.0,0.0,NaN,0.05,2016-11-30,AK00400001,INTCK7.5K,1.0,24999.0,0.0,NaN,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,AK00400001,18MBSCD10K,1.0,24999.0,18.0,M,0.55,2016-11-30,AK00400001,18MBSCD50K,25000.0,99999.0,18.0,M,0.65,2016-11-30,AK00400001,24MBSCD10K,1.0,24999.0,24.0,M,0.75,2016-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,AK00400002,AK00400002,36MCD25K,500.0,9999999.0,36.0,M,1.49,2016-11-30,AK00400002,FIXIRA50K,500.0,9999999.0,0.0,NaN,0.75,2016-11-30,AK00400002,FIXIRA0K,500.0,9999999.0,0.0,NaN,0.750,2016-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,AL00100010,AL00100010,36MCD25K,500.0,9999999.0,36.0,M,0.41,2016-11-29,AL00100010,FIXIRA50K,500.0,9999999.0,0.0,NaN,0.16,2016-11-29,AL00100010,FIXIRA0K,500.0,9999999.0,0.0,NaN,0.160,2016-11-29,AL00100010,INTCK5K,25.0,9999999.0,0.0,NaN,0.02,2016-11-29,AL00100010,INTCK7.5K,25.0,9999999.0,0.0,NaN,0.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
gc.collect()

0

In [ ]:
wide12 = pd.merge(wide1, wide2, on=['acct_nbr_loc'], how ='inner', validate="1:1")
wide123 = pd.merge(wide12, wide3, on=['acct_nbr_loc'], how ='inner', validate="1:1")
wide1234 = pd.merge(wide123, wide4, on=['acct_nbr_loc'], how ='inner', validate="1:1")
wide12345 = pd.merge(wide1234, wide5, on=['acct_nbr_loc'], how ='inner', validate="1:1")
wide12345

,acct_nbr_loc,24MCD10K_acct_nbr_rt,24MCD10K_productcode,24MCD10K_tierminimum,24MCD10K_tiermaximum,24MCD10K_termlength,24MCD10K_termfrequency,24MCD10K_rate,24MCD10K_surveydate,24MCD100K_acct_nbr_rt,24MCD100K_productcode,24MCD100K_tierminimum,24MCD100K_tiermaximum,24MCD100K_termlength,24MCD100K_termfrequency,24MCD100K_rate,24MCD100K_surveydate,30MCD10K_acct_nbr_rt,30MCD10K_productcode,30MCD10K_tierminimum,30MCD10K_tiermaximum,30MCD10K_termlength,30MCD10K_termfrequency,30MCD10K_rate,30MCD10K_surveydate,24MCD40K_acct_nbr_rt,24MCD40K_productcode,24MCD40K_tierminimum,24MCD40K_tiermaximum,24MCD40K_termlength,24MCD40K_termfrequency,24MCD40K_rate,24MCD40K_surveydate,30MCD40K_acct_nbr_rt,30MCD40K_productcode,30MCD40K_tierminimum,30MCD40K_tiermaximum,30MCD40K_termlength,30MCD40K_termfrequency,30MCD40K_rate,...,11MBSCD0K_acct_nbr_rt,11MBSCD0K_productcode,11MBSCD0K_tierminimum,11MBSCD0K_tiermaximum,11MBSCD0K_termlength,11MBSCD0K_termfrequency,11MBSCD0K_rate,11MBSCD0K_surveydate,14DBSCD0K_acct_nbr_rt,14DBSCD0K_productcode,14DBSCD0K_tierminimum,14DBSCD0K_tiermaximum,14DBSCD0K_termlength,14DBSCD0K_termfrequency,14DBSCD0K_rate,14DBSCD0K_surveydate,21DBSCD0K_acct_nbr_rt,21DBSCD0K_productcode,21DBSCD0K_tierminimum,21DBSCD0K_tiermaximum,21DBSCD0K_termlength,21DBSCD0K_termfrequency,21DBSCD0K_rate,21DBSCD0K_surveydate,BRELMM0K_acct_nbr_rt,BRELMM0K_productcode,BRELMM0K_tierminimum,BRELMM0K_tiermaximum,BRELMM0K_termlength,BRELMM0K_termfrequency,BRELMM0K_rate,BRELMM0K_surveydate,24MVIRA0K_acct_nbr_rt,24MVIRA0K_productcode,24MVIRA0K_tierminimum,24MVIRA0K_tiermaximum,24MVIRA0K_termlength,24MVIRA0K_termfrequency,24MVIRA0K_rate,24MVIRA0K_surveydate
0,AK00100003,AK00100003,24MCD10K,10000.0,99999.0,24.0,M,1.34,2016-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,AK00100003,30MCD10K,10000.0,99999.0,30.0,M,1.34,2016-11-30,AK00100003,24MCD40K,10000.0,99999.0,24.0,M,1.34,2016-11-30,AK00100003,30MCD40K,10000.0,99999.0,30.0,M,1.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,AK00100006,AK00100006,24MCD10K,500.0,24999.0,24.0,M,0.20,2016-11-30,AK00100006,24MCD100K,100000.0,499999.0,24.0,M,0.30,2016-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,AK00400001,AK00400001,24MCD10K,1.0,24999.0,24.0,M,0.75,2016-11-30,AK00400001,24MCD100K,100000.0,249999.0,24.0,M,0.85,2016-11-30,AK00400001,30MCD10K,1.0,24999.0,30.0,M,0.75,2016-11-30,AK00400001,24MCD40K,25000.0,99999.0,24.0,M,0.80,2016-11-30,AK00400001,30MCD40K,25000.0,99999.0,30.0,M,0.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,AK00400002,AK00400002,24MCD10K,500.0,9999999.0,24.0,M,1.24,2016-11-30,AK00400002,24MCD100K,500.0,9999999.0,24.0,M,1.24,2016-11-30,AK00400002,30MCD10K,500.0,9999999.0,30.0,M,1.24,2016-11-30,AK00400002,24MCD40K,500.0,9999999.0,24.0,M,1.24,2016-11-30,AK00400002,30MCD40K,500.0,9999999.0,30.0,M,1.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,AL00100010,AL00100010,24MCD10K,500.0,9999999.0,24.0,M,0.31,2016-11-29,AL00100010,24MCD100K,500.0,9999999.0,24.0,M,0.31,2016-11-29,AL00100010,30MCD10K,500.0,9999999.0,30.0,M,0.35,2016-11-29,AL00100010,24MCD40K,500.0,9999999.0,24.0,M,0.31,2016-11-29,AL00100010,30MCD40K,500.0,9999999.0,30.0,M,0.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
Finalsheet = reduce(lambda x,y: pd.merge(x,y, on='acct_nbr_loc', how='left'), [wide12345,unique_Deposit_InstitutionDetails122016])
print (len(Finalsheet.index))
Finalsheet.head(20)

8557


,acct_nbr_loc,24MCD10K_acct_nbr_rt,24MCD10K_productcode,24MCD10K_tierminimum,24MCD10K_tiermaximum,24MCD10K_termlength,24MCD10K_termfrequency,24MCD10K_rate,24MCD10K_surveydate,24MCD100K_acct_nbr_rt,24MCD100K_productcode,24MCD100K_tierminimum,24MCD100K_tiermaximum,24MCD100K_termlength,24MCD100K_termfrequency,24MCD100K_rate,24MCD100K_surveydate,30MCD10K_acct_nbr_rt,30MCD10K_productcode,30MCD10K_tierminimum,30MCD10K_tiermaximum,30MCD10K_termlength,30MCD10K_termfrequency,30MCD10K_rate,30MCD10K_surveydate,24MCD40K_acct_nbr_rt,24MCD40K_productcode,24MCD40K_tierminimum,24MCD40K_tiermaximum,24MCD40K_termlength,24MCD40K_termfrequency,24MCD40K_rate,24MCD40K_surveydate,30MCD40K_acct_nbr_rt,30MCD40K_productcode,30MCD40K_tierminimum,30MCD40K_tiermaximum,30MCD40K_termlength,30MCD40K_termfrequency,30MCD40K_rate,...,11MBSCD0K_surveydate,14DBSCD0K_acct_nbr_rt,14DBSCD0K_productcode,14DBSCD0K_tierminimum,14DBSCD0K_tiermaximum,14DBSCD0K_termlength,14DBSCD0K_termfrequency,14DBSCD0K_rate,14DBSCD0K_surveydate,21DBSCD0K_acct_nbr_rt,21DBSCD0K_productcode,21DBSCD0K_tierminimum,21DBSCD0K_tiermaximum,21DBSCD0K_termlength,21DBSCD0K_termfrequency,21DBSCD0K_rate,21DBSCD0K_surveydate,BRELMM0K_acct_nbr_rt,BRELMM0K_productcode,BRELMM0K_tierminimum,BRELMM0K_tiermaximum,BRELMM0K_termlength,BRELMM0K_termfrequency,BRELMM0K_rate,BRELMM0K_surveydate,24MVIRA0K_acct_nbr_rt,24MVIRA0K_productcode,24MVIRA0K_tierminimum,24MVIRA0K_tiermaximum,24MVIRA0K_termlength,24MVIRA0K_termfrequency,24MVIRA0K_rate,24MVIRA0K_surveydate,inst_nm,uninumbr,address,city,state,lon,lat
0,AK00100003,AK00100003,24MCD10K,10000.0,99999.0,24.0,M,1.340,2016-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,AK00100003,30MCD10K,10000.0,99999.0,30.0,M,1.340,2016-11-30,AK00100003,24MCD40K,10000.0,99999.0,24.0,M,1.340,2016-11-30,AK00100003,30MCD40K,10000.0,99999.0,30.0,M,1.340,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Alaska USA Federal Credit Union,29975.0,4000 Credit Union Dr,Anchorage,AK,-149.889475,61.183923
1,AK00100006,AK00100006,24MCD10K,500.0,24999.0,24.0,M,0.200,2016-11-30,AK00100006,24MCD100K,100000.0,499999.0,24.0,M,0.300,2016-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Northrim Bank,46457.0,3111 C St,Anchorage,AK,-149.885906,61.191917
2,AK00400001,AK00400001,24MCD10K,1.0,24999.0,24.0,M,0.750,2016-11-30,AK00400001,24MCD100K,100000.0,249999.0,24.0,M,0.850,2016-11-30,AK00400001,30MCD10K,1.0,24999.0,30.0,M,0.750,2016-11-30,AK00400001,24MCD40K,25000.0,99999.0,24.0,M,0.800,2016-11-30,AK00400001,30MCD40K,25000.0,99999.0,30.0,M,0.800,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,First Bank,9320.0,331 Dock St,Ketchikan,AK,-131.653466,55.344615
3,AK00400002,AK00400002,24MCD10K,500.0,9999999.0,24.0,M,1.240,2016-11-30,AK00400002,24MCD100K,500.0,9999999.0,24.0,M,1.240,2016-11-30,AK00400002,30MCD10K,500.0,9999999.0,30.0,M,1.240,2016-11-30,AK00400002,24MCD40K,500.0,9999999.0,24.0,M,1.240,2016-11-30,AK00400002,30MCD40K,500.0,9999999.0,30.0,M,1.240,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Tongass Federal Credit Union,19738.0,2000 Tongass Ave,Ketchikan,AK,-131.676110,55.351225
4,AL00100010,AL00100010,24MCD10K,500.0,9999999.0,24.0,M,0.310,2016-11-29,AL00100010,24MCD100K,500.0,9999999.0,24.0,M,0.310,2016-11-29,AL00100010,30MCD10K,500.0,9999999.0,30.0,M,0.350,2016-11-29,AL00100010,24MCD40K,500.0,9999999.0,24.0,M,0.310,2016-11-29,AL00100010,30MCD40K,500.0,9999999.0,30.0,M,0.350,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Commonwealth National Bank,15447.0,2214 St Stephens Rd,M

In [ ]:
Finalsheet.to_csv("/content/drive/My Drive/Finalsheet.csv", index = False, sep='|',mode = 'w', encoding='utf-8')

In [ ]:
Finalsheet.shape

(8557, 5992)